# Smart Contract Crash Course 

## Basic interaction with `geth`

The most common client lib to interface with ethereum execution clients (`geth`,`ganache`,...) is **web3**.
* [web3.js](https://github.com/ethereum/web3.js/)
    - node.js version
    - [API](https://web3js.readthedocs.io/)
* [web3py](https://github.com/ethereum/web3.py)
    - python version
    - as I prefer python, we will use this one ;)
    - [API](https://web3py.readthedocs.io/)
    
For this tutorial we will only interact with `geth`: 
* [geth console](https://geth.ethereum.org/docs/interface/javascript-console) has a link to the supported API via the geth console, which should be the "full" `web3.js` API.
* [geth cli options](https://geth.ethereum.org/docs/interface/command-line-options) only needed when running geth from the command line

## Connect via RPC/HTTP

Note that in our case `geth` is running in a private PoA chain setup.
Therefore, some configuration parameters are different. 

https://web3py.readthedocs.io/en/stable/middleware.html#geth-style-proof-of-authority

In [1]:
import web3
from web3.middleware import geth_poa_middleware

w3 = web3.Web3(web3.Web3.HTTPProvider("http://geth-client-cnt:8545"))
# check if connection was successful
assert w3.is_connected()

w3.middleware_onion.inject(geth_poa_middleware, layer=0)

In [2]:
w3.geth.admin.node_info()

AttributeDict({'id': '76a8c5bca218e0ecfca09818ff4112faaf6aa1c9af9947ef3518e12127fa2b1b',
 'name': 'Geth/bob/v1.10.3-stable-991384a7/linux-amd64/go1.21.5',
 'enode': 'enode://d8ed6c277b0878cad10cc7a53daa8efbfe4a04b73016a484ef1df3eca7bd44d3d06c0707030ec5fbfcad0f8350e2a79a8491f5b6379e9cf4f5fcd04e8c6fa03d@127.0.0.1:30303?discport=0',
 'enr': 'enr:-Ja4QGjX13WHXG9FOVyLyObNbiN3-ZbAwp4ECD9rmp5J-CHySk0NqpsOZMf9XGgvvJvq6IDNeqA4u3O_csPcvlAdCQoCg2V0aMfGhL8tJ2SAgmlkgnY0gmlwhH8AAAGJc2VjcDI1NmsxoQPY7Wwnewh4ytEMx6U9qo77_koEtzAWpITvHfPsp71E04RzbmFwwIN0Y3CCdl8',
 'ip': '127.0.0.1',
 'ports': AttributeDict({'discovery': 0, 'listener': 30303}),
 'listenAddr': '[::]:30303',
 'protocols': AttributeDict({'eth': AttributeDict({'network': 20240101,
   'difficulty': 3235,
   'genesis': '0x6bec1279ca6efde179db76897011152e3b1d4c89f11298b501a501511fa016b8',
   'config': AttributeDict({'chainId': 20240101,
    'homesteadBlock': 0,
    'eip150Block': 0,
    'eip150Hash': '0x000000000000000000000000000000000000000000

In [3]:
# display the client version of the node we are connected to
w3.client_version

'Geth/bob/v1.10.3-stable-991384a7/linux-amd64/go1.21.5'

In [4]:
# display network ID of client you are connected to
w3.net.version

'20240101'

In [5]:
w3.eth.chain_id

20240101

In [6]:
# Display number of connected peers should be 1
w3.net.peer_count

1

In [7]:
# check if node is "up-to-date" with blockchain, 
# i.e., not syncing anymore
w3.eth.syncing

False

In [8]:
w3.eth.mining

False

## Connect to enode with IP

To connect to a geth node a `enode` ulr is required. This contains a public key of this node as well as its domain name (or IP) and prot. 

The enode of our server is: 
`enode://fd90f39e40633934f1613ab08bd4917d3aa28abba28cc4748b544650cfffe1ab40890f4a9d85ca9c0cf28845633291ca06a5716b86f5232b32374b82e2e3cff6@eth-smart.secenv:30303?discport=0`

In [9]:
enode = "enode://fd90f39e40633934f1613ab08bd4917d3aa28abba28cc4748b544650cfffe1ab40890f4a9d85ca9c0cf28845633291ca06a5716b86f5232b32374b82e2e3cff6@eth-smart.secenv:30303?discport=0"

In [10]:
#enode = "enode://fd90f39e40633934f1613ab08bd4917d3aa28abba28cc4748b544650cfffe1ab40890f4a9d85ca9c0cf28845633291ca06a5716b86f5232b32374b82e2e3cff6@10.81.0.38:30303?discport=0"

In [11]:
w3.geth.admin.add_peer(enode)

True

In [12]:
w3.net.peer_count

1

In [13]:
w3.geth.admin.peers()

[AttributeDict({'enode': 'enode://fd90f39e40633934f1613ab08bd4917d3aa28abba28cc4748b544650cfffe1ab40890f4a9d85ca9c0cf28845633291ca06a5716b86f5232b32374b82e2e3cff6@10.81.0.38:30303?discport=0',
  'id': '2af0d687348cffa0a8d8e827abd4e1e5f2c68e4beca2dfaa768856a9dbba522e',
  'name': 'Geth/alice/v1.10.3-stable-991384a7/linux-amd64/go1.21.5',
  'caps': ['eth/65', 'eth/66', 'snap/1'],
  'network': AttributeDict({'localAddress': '172.22.0.4:36870',
   'remoteAddress': '10.81.0.38:30303',
   'inbound': False,
   'trusted': False,
   'static': True}),
  'protocols': AttributeDict({'eth': AttributeDict({'version': 66,
    'difficulty': 3233,
    'head': '0x00b6fafac72ce9c056c9286aa3d7c1fa5bfa9274170b103ac11c5d4a0ddbcce3'}),
   'snap': AttributeDict({'version': 1})})})]

To manually extract the enode from a running geth node, cat the `nodekey` file which contains the private key  and use the `bootnode` tool to generate the associated public key, i.e., enode. (see )

```bash
$ cat $DATADIR/geth/nodekey
$ bootnode -nodekeyhex $(cat $DATADIR/geth/nodekey) -writeaddress
```

In [14]:
w3.eth.syncing

False

## Blocks

In [15]:
# return current blockchain head of node
w3.eth.block_number

1618

In [16]:
# should be greater
!sleep 16
w3.eth.block_number

1619

In [17]:
# block 0 ist the genesis block of this testnet blockchain
genesis_block = w3.eth.get_block(0)
genesis_block

AttributeDict({'difficulty': 1,
 'proofOfAuthorityData': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000f78342e0cdb2188e3b5c9663b1c6d6d5af68bc970000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
 'gasLimit': 15000000,
 'gasUsed': 0,
 'hash': HexBytes('0x6bec1279ca6efde179db76897011152e3b1d4c89f11298b501a501511fa016b8'),
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
 'miner': '0x00000000000

As you see, the genesis block is special as it is the only block with no parent hash:

In [18]:
genesis_block['number']

0

In [19]:
genesis_block['parentHash']

HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000')

The [genesis.json](../genesis_config/go-ethereum/berlin/genesis.json) file shows the node(s) that are allowed to mine blocks and the accounts that have been seeded with coins. 
The client we are currently connected to is not part of the PoA nodes that are allowed to create new blocks:

## Accounts and balances

Accounts are your **external accounts** consisting of public and private keys. 
The ethereum address are the last 40 hex characters (20 bytes) of the hash (Keccak-256) of the public key prefixed with `0x`

For the exercise you will recieve your personal account keypair which you have to copy in your `geth` folder to import it. 

In [20]:
!pwd

/smartenv


In [21]:
!ls 

Containerfile	       geth		 sccc-geth.ipynb   solutions
connection_test.ipynb  requirements.txt  sccc-intro.ipynb  util
entrypoint.sh	       sccc-anvil.ipynb  setup		   venv


In [25]:
KEYSTORE_DIR = "/keystore"

In [26]:
!ls {KEYSTORE_DIR}

UTC--2024-01-17T17-06-29.575108Z--53B30788b6a47261be56a851C22B155cd3b84735


In [27]:
w3.eth.accounts

['0x53B30788b6a47261be56a851C22B155cd3b84735']

The default address (also used to receive rewards from mining blocks) is usually the first in the list of accounts and usually also accessible via `w3.eth.coinbase`. It is good practise to specifically assing a `default_account` some functions might cause problems if this is not explicitly assigned.

In [29]:
w3.eth.default_account = w3.eth.accounts[0]
w3.eth.default_account 

'0x53B30788b6a47261be56a851C22B155cd3b84735'

In [51]:
# show balance in wei
w3.eth.get_balance(w3.eth.default_account)

14999979000000000000

In [58]:
# show balance in ether
def getBalance(address):
    return w3.from_wei(w3.eth.get_balance(address),'ether')

In [57]:
initial_balance = getBalance(w3.eth.accounts[0]) 
assert initial_balance > 0 
initial_balance

Decimal('14.999979')

### Create New Account

Create a new account with no password using geth directly:

In [40]:
!geth --maxpeers 0 --datadir=/ --password=/dev/null account new

INFO [01-17|20:09:55.209] Maximum peer count                       ETH=0 LES=0 total=0
INFO [01-17|20:09:55.212] Smartcard socket not found, disabling    err="stat /run/pcscd/pcscd.comm: no such file or directory"

Your new key was generated

Public address of the key:   0x06F2542FA3D541f33A7ea33772666dA16Cb2e86a
Path of the secret key file: /keystore/UTC--2024-01-17T20-09-55.212535092Z--06f2542fa3d541f33a7ea33772666da16cb2e86a

- You can share your public address with anyone. Others need it to interact with you.
- You must NEVER share the secret key with anyone! The key controls access to your funds!
- You must BACKUP your key file! Without the key, it's impossible to access account funds!
- You must REMEMBER your password! Without the password, it's impossible to decrypt the key!



It requires some time till geth picks up the new key:

In [41]:
!sleep 5

In [42]:
w3.eth.accounts

['0x53B30788b6a47261be56a851C22B155cd3b84735',
 '0x06F2542FA3D541f33A7ea33772666dA16Cb2e86a']

## Payment Transaction and Gas

**Note:** When connected to `geth` or `parity` you have to unlock an account first to send a transaction since the private keys are stored encrypted per default and protected with a password. In `anvil` this is not necessary since it is a development environment.

Per default unlocking an account via HTTP is not allowed in `geth` so this has to be explicitly activated 
with `--allow-insecure-unlock` when starting `geth`. Do not use this flag in production! 

In [44]:
# Parameters are:
# * account
# * password
# * duration of unlock (if 0 then forever)
w3.geth.personal.unlock_account(w3.eth.accounts[0],"password",0)

True

In [63]:
blkNumber_before = w3.eth.block_number
blkNumber_before

1671

In [71]:
w3.eth.send_transaction({'from':w3.eth.accounts[0], 
                        'to':w3.eth.accounts[1], 
                        'value':10**18})

HexBytes('0xd942d23fe537f8632d585a81c8df175f9c5efd8962d1bed8d7c85e989d729ffa')

In [72]:
!sleep 13
blkNumber_after = w3.eth.block_number
assert blkNumber_before < blkNumber_after
blkNumber_after

1675

In [73]:
# the balance is reduced by more than 1 ether
# Why do you think this is the case?
getBalance(w3.eth.accounts[0])

Decimal('12.999937')

In [74]:
new_account_balance = getBalance(w3.eth.accounts[1])
assert int(new_account_balance) >= 1
int(new_account_balance)

3

In [81]:
# get first (and only) transaction in the previously mined block
tx = w3.eth.get_transaction_by_block(blkNumber_after,0)
tx 

AttributeDict({'blockHash': HexBytes('0xd7e20448f21e3cebc1cfc119ee07ff2e443a00c3e2179bdba517bcbe3b497271'),
 'blockNumber': 1675,
 'from': '0x53B30788b6a47261be56a851C22B155cd3b84735',
 'gas': 121000,
 'gasPrice': 1000000000,
 'hash': HexBytes('0xd942d23fe537f8632d585a81c8df175f9c5efd8962d1bed8d7c85e989d729ffa'),
 'input': HexBytes('0x'),
 'nonce': 2,
 'to': '0x06F2542FA3D541f33A7ea33772666dA16Cb2e86a',
 'transactionIndex': 0,
 'value': 1000000000000000000,
 'type': 0,
 'v': 40480238,
 'r': HexBytes('0x70a92ba765eb04ade3877d490eb0abc3689da4ee2e48a16bc9acb6c5e71611be'),
 's': HexBytes('0x0e1c9706fcfd20fca4d8dc0cf9c77fba04fef754a558eaae4c0a9d2b6d3a7583')})

In [83]:
# get block and see the transaction id and gas used below
blk = w3.eth.get_block(blkNumber_after)
blk

AttributeDict({'difficulty': 2,
 'proofOfAuthorityData': HexBytes('0xd883010a03846765746888676f312e32312e35856c696e7578000000000000004fe6c9143efa2de28974e936479ca732a2e79ae233beaec51d8e1fa3e42c506d352c4ffa9b0b265a814cfc435b2e317cdb6a27070e4634ba7c377485e37d609801'),
 'gasLimit': 8000000,
 'gasUsed': 21000,
 'hash': HexBytes('0xd7e20448f21e3cebc1cfc119ee07ff2e443a00c3e2179bdba517bcbe3b497271'),
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
 'miner': '0x0000000000000000000000000000000000000000',
 'mix

In [84]:
# get the transaction ID of the first (and only) transaction in the block
blk["transactions"][0].hex()

'0xd942d23fe537f8632d585a81c8df175f9c5efd8962d1bed8d7c85e989d729ffa'

In [85]:
tx["hash"].hex() # hash of the transaction

'0xd942d23fe537f8632d585a81c8df175f9c5efd8962d1bed8d7c85e989d729ffa'

In [86]:
# Query the transaction by ID, also possible
w3.eth.get_transaction(blk["transactions"][0].hex())

AttributeDict({'blockHash': HexBytes('0xd7e20448f21e3cebc1cfc119ee07ff2e443a00c3e2179bdba517bcbe3b497271'),
 'blockNumber': 1675,
 'from': '0x53B30788b6a47261be56a851C22B155cd3b84735',
 'gas': 121000,
 'gasPrice': 1000000000,
 'hash': HexBytes('0xd942d23fe537f8632d585a81c8df175f9c5efd8962d1bed8d7c85e989d729ffa'),
 'input': HexBytes('0x'),
 'nonce': 2,
 'to': '0x06F2542FA3D541f33A7ea33772666dA16Cb2e86a',
 'transactionIndex': 0,
 'value': 1000000000000000000,
 'type': 0,
 'v': 40480238,
 'r': HexBytes('0x70a92ba765eb04ade3877d490eb0abc3689da4ee2e48a16bc9acb6c5e71611be'),
 's': HexBytes('0x0e1c9706fcfd20fca4d8dc0cf9c77fba04fef754a558eaae4c0a9d2b6d3a7583')})

In [87]:
gasUsed = blk["gasUsed"] # gas used within block
gasUsed

21000

In [88]:
# estimate the gas price of an transaction without executing it
w3.eth.estimate_gas({'from':w3.eth.accounts[0], 
                    'to':w3.eth.accounts[1], 
                    'value':10**18})

21000

The estimate yields the same result as the actual gas cost. 

In [89]:
gasPrice = tx["gasPrice"] # The actual gas price attached to the tx
gasPrice

1000000000

In [91]:
# conversion between gas and ether specified per transaction
# i.e., how much wei a transaction is willing to pay for 
assert gasPrice == w3.eth.gas_price # default value

In [92]:
# gasUsed(by the transaction) * gasPrice(specified in tx)
# gives the wei it costs to send it 
# times 10^18 gives the ether
tx_costs = (gasUsed * gasPrice) * 10**-18
tx_costs 

2.1000000000000002e-05

The gas accounts for the missing cost on the sender side: 